In [1]:
import copy
from itertools import combinations

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets import interact
from numpy import arange, mean
from pylab import meshgrid, cm
import numpy as np
import random as rand
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def function(a,b):
    index = func_selector.options.index(func_selector.value)
    if index == 0:
        return (a-2)**4+(a-2*b)**2
    if index == 1:
        return -12*b+4*a**2+4*b**2-4*a*b
    if index == 2:
        return 8*a**2+4*a*b+5*b**2

In [3]:
def plot(population):
    x = arange(-5,8,0.1)
    y = arange(-5,8,0.1)
    X,Y = meshgrid(x, y)
    Z = function(X, Y)
    fig, ax = plt.subplots(figsize=(10,10))
    im = ax.imshow(Z,cmap=cm.viridis,origin='lower',extent=(-5,8,-5,8))
    cset = ax.contour(Z,arange(0,500,5),linewidths=0.5,cmap=cm.Greys,extent=(-5,8,-5,8))
    ax.plot(population[0],population[1],"+",color='red')
    plt.show()

In [4]:
def sort(Pbest, r, n):
    for i in range(1, n):
        for j in range(0, n - 1):
                tempRes = r[j]
                r[j] = r[j + 1]
                r[j + 1] = tempRes
                
                tempX = Pbest[0][j]
                Pbest[0][j] = Pbest[0][j + 1]
                Pbest[0][j + 1] = tempX

                tempY = Pbest[1][j]
                Pbest[1][j] = Pbest[1][j + 1]
                Pbest[1][j + 1] = tempY

In [5]:
def vect_velocity(n, a, Pbest, Gbest, v):
    for i in range(n):
        # X
        v[0][i] = 0.7 * v[0][i] + (Pbest[0][i] - a[0][i]) * rand.random() * 1.47 + (
                    Gbest[0][0] - a[0][i]) * rand.random() * 1.47
        a[0][i] = a[0][i] + v[0][i]
        # Y
        v[1][i] = 0.7 * v[1][i] + (Pbest[1][i] - a[1][i]) * rand.random() * 1.47 + (
                    Gbest[0][1] - a[1][i]) * rand.random() * 1.47
        a[1][i] = a[1][i] + v[1][i]

In [9]:
def PSO():
    num_variables = 2
    n = population_size.value
    a = np.empty((num_variables, n))
    v = np.empty((num_variables, n))
    Pbest = np.empty((num_variables, n))
    Gbest = np.empty((1, 2))
    r = np.empty((n))

    for i in range(0, num_variables):
        for j in range(0, n):
            Pbest[i][j] = rand.randint(-2, 5)
            a[i][j] = Pbest[i][j]
            v[i][j] = 0

    for i in range(0, n):
        r[i] = function(a[0][i], a[1][i])

    sort(Pbest, r, n)

    Gbest[0][0] = Pbest[0][0]
    Gbest[0][1] = Pbest[1][0]

    generation = 0

    history.append((copy.copy(a[0]), copy.copy(a[1])))
    while (generation < steps_count.value):
        progress.value=generation+1
        for i in range(n):
            if (function(a[0][i], a[1][i]) < function(Pbest[0][i], Pbest[1][i])):
                Pbest[0][i] = a[0][i]
                Pbest[1][i] = a[1][i]
            if (function(Pbest[0][i], Pbest[1][i]) < function(Gbest[0][0], Gbest[0][1])):
                Gbest[0][0] = Pbest[0][i]
                Gbest[0][1] = Pbest[1][i]
            vect_velocity(n, a, Pbest, Gbest, v)
        generation = generation + 1
#         print('Generacion: ' + str(generation) + ' - - - Gbest: ' + str(Gbest))
        history.append((copy.copy(a[0]), copy.copy(a[1])))
    return Gbest[0]

In [10]:
func_selector = widgets.Dropdown(
    options=['(a-2)^4+(a-2b)^2 [2;1]', '-12b+4a^2+4b^2-4ab [1;2]', '8a^2+4ab+5b^2 [0;0]'],
    description='Функция:'
)

population_size = widgets.IntText(
    value=50,
    description='Размер популяции:',
    disabled=False
)
steps_count = widgets.IntText(
    value=50,
    description='Кол-во шагов:',
    disabled=False
)
start = widgets.Button(
    description='Старт'
)
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Прогресс:',
    bar_style='',
    orientation='horizontal'
)

def build_plot(step):
    plot(history[step-1])
history =[]

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    step=1,
    description='Прогресс:',
    bar_style='',
    orientation='horizontal'
)

def on_button_clicked(b):
    display(progress)
    progress.max=steps_count.value
    global history
    history = []
    res = PSO()
    interact(build_plot,step=widgets.IntSlider(min=1,max=len(history),step=1,value=1,description="Шаг:"))
    print(f'Результат оптимизации: {res[0]:.2f} {res[1]:.2f}')

display(func_selector)
start.on_click(on_button_clicked)
display(population_size)
display(steps_count)
display(start)

Dropdown(description='Функция:', options=('(a-2)^4+(a-2b)^2 [2;1]', '-12b+4a^2+4b^2-4ab [1;2]', '8a^2+4ab+5b^2…

IntText(value=50, description='Размер популяции:')

IntText(value=50, description='Кол-во шагов:')

Button(description='Старт', style=ButtonStyle())

IntProgress(value=0, description='Прогресс:')

interactive(children=(IntSlider(value=1, description='Шаг:', max=51, min=1), Output()), _dom_classes=('widget-…

Результат оптимизации: 2.00 1.00


IntProgress(value=50, description='Прогресс:', max=50)

interactive(children=(IntSlider(value=1, description='Шаг:', max=51, min=1), Output()), _dom_classes=('widget-…

Результат оптимизации: 0.00 0.00
